# Notebook 03: Inferencia del Modelo y Predicción 2024
## Objetivo: Generar el pronóstico de demanda final para el año 2024.

## Metodología:

- Cargar el historial completo de datos (data/raw/).

- Cargar los hiperparámetros del modelo validado (models/gbr_model.joblib).

- Re-entrenar un modelo GBR final usando TODOS los datos históricos (2012-2023) y los hiperparámetros guardados.

- Crear un scaffold (andamio) de fechas para 2024.

- Generar las predicciones para 2024.

- Guardar el pronóstico final en data/processed/.

## 0. Configuración e Importación

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import GradientBoostingRegressor

# Configuración de visualización
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 7)

## 1. Definir Función de Creación de Features
Esta función debe ser idéntica a la usada en el notebook 01-model_core.ipynb para asegurar consistencia.

In [ ]:
def create_features(data):
    """Crea características de series de tiempo en el dataframe."""
    data = data.sort_values(['prod_id', 'fecha']).copy()
    
    # Features de Calendario
    data['year'] = data['fecha'].dt.year
    data['month'] = data['fecha'].dt.month
    
    # Lags Anuales (robustez para predicción a 12 meses)
    data['lag_12'] = data.groupby('prod_id')['ventas'].shift(12)
    data['lag_13'] = data.groupby('prod_id')['ventas'].shift(13)
    data['lag_24'] = data.groupby('prod_id')['ventas'].shift(24)
    
    # Media móvil sobre el lag para suavizar
    data['rolling_mean_3_lag12'] = data.groupby('prod_id')['lag_12'].transform(lambda x: x.rolling(3).mean())
    
    # Lag del precio (usamos el precio de hace un año como proxy)
    data['precio_lag_12'] = data.groupby('prod_id')['precio_promedio'].shift(12)
    
    return data

## 2. Cargar Datos e Hiperparámetros
Cargamos el historial completo y los parámetros del modelo que validamos en el notebook 01.

In [ ]:
# --- 1. Cargar TODOS los datos (historial completo) ---
path_datos = '../data/raw/demanding_forecast.csv'
df = pd.read_csv(path_datos)
df['fecha'] = pd.to_datetime(df['fecha'])

print(f"Historial completo cargado. {len(df)} filas.")

# --- 2. Cargar los hiperparámetros del modelo validado ---
path_modelo_validado = '../models/gbr_model.joblib'
gbr_validado = joblib.load(path_modelo_validado)
model_params = gbr_validado.get_params()

print(f"Hiperparámetros cargados desde: {path_modelo_validado}")
print(f"Learning rate: {model_params['learning_rate']}, N_estimators: {model_params['n_estimators']}, Max_depth: {model_params['max_depth']}")

## 3. Re-entrenamiento del Modelo Final
Ahora entrenamos un modelo nuevo (gbr_final) usando los parámetros óptimos, pero esta vez con todos los datos hasta Diciembre 2023.

In [ ]:
# --- 3. Crear features en TODO el historial ---
df_model = create_features(df)
df_model = df_model.dropna()

# --- 4. Definir el set de entrenamiento FINAL ---
# Usamos TODOS los datos hasta el final de 2023
features = ['month', 'year', 'prod_id', 
            'lag_12', 'lag_13', 'lag_24', 
            'rolling_mean_3_lag12', 'precio_lag_12']
target = 'ventas'

train_full = df_model[df_model['year'] <= 2023]
print(f"Re-entrenando con {len(train_full)} filas (historial completo 2012-2023).")

# --- 5. Re-entrenar el modelo FINAL ---
gbr_final = GradientBoostingRegressor(**model_params)

# Quitamos 'validation_fraction' y 'n_iter_no_change' ya que no estamos validando,
# sino entrenando con todo.
gbr_final.set_params(validation_fraction=None, n_iter_no_change=None)

gbr_final.fit(train_full[features], train_full[target])

print("Modelo final re-entrenado y listo para inferencia.")

## 4. Crear Scaffold (Andamio) 2024 y Predecir
Construimos el esqueleto de 2024, lo unimos al historial para poder calcular los lags, y luego filtramos solo 2024 para predecir.

In [ ]:
# --- 6. Crear scaffold de 2024 y predecir ---
future_dates = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
prod_ids = df['prod_id'].unique()

df_future_rows = []
for pid in prod_ids:
    for date in future_dates:
        df_future_rows.append({
            'fecha': date, 'prod_id': pid,
            'ventas': np.nan, 'precio_promedio': np.nan
        })

df_future = pd.DataFrame(df_future_rows)

# Concatenar historial + futuro
df_full_history = pd.concat([df, df_future], axis=0) 

# Aplicar features al DF combinado
df_full_features = create_features(df_full_history) 

# Filtrar solo el 2024 (que ahora tiene features como lag_12)
df_2024 = df_full_features[df_full_features['fecha'].dt.year == 2024].copy()

# Predecir
# (Puede haber nulos en features si un producto es muy nuevo, por seguridad llenamos con 0)
df_2024[features] = df_2024[features].fillna(0)
df_2024['prediccion_ventas'] = gbr_final.predict(df_2024[features]) # Usar gbr_final

print("Predicciones 2024 generadas.")

5. Guardar Salida Procesada
Este es el archivo final que el negocio consumirá. Se guarda en data/processed/.

In [ ]:
# --- 7. Guardar Salida Procesada ---
path_salida = '../data/processed/predicciones_demanda_2024.csv'
output_cols = ['fecha', 'prod_id', 'prediccion_ventas']
df_2024[output_cols].to_csv(path_salida, index=False)

print(f"Predicciones 2024 generadas y guardadas en '{path_salida}'")
print(df_2024[output_cols].head())

6. Conclusión del Proyecto
Este notebook completa el pipeline de modelado.

Acciones para el Negocio:

Tomar el archivo predicciones_demanda_2024.csv como el pronóstico base.

Sumarle el stock de seguridad calculado en el notebook 02-model_risk_gpd.ipynb (ej. + 919 unidades).

Stock Total a Planificar = prediccion_ventas + stock_de_seguridad_total